In [1]:
import edt
import SimpleITK as sitk
from glob import glob
from pathlib import Path
import json
import pandas as pd
import os
import tqdm
import numpy as np

train_vessel_dir = "/work/vig/Datasets/aneurysm/internal_train/crop_0.4_vessel"

test_vessel_dir = "/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel"
external_vessel_dir = "/work/vig/Datasets/aneurysm/external/crop_0.4_vessel"
target_train_vessel_dir = (
    "/work/vig/Datasets/aneurysm/internal_train/crop_0.4_vessel_edt/"
)
target_test_vessel_dir = (
    "/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_edt/"
)

hospital_vessel_dir = "/work/vig/Datasets/aneurysm/hospital/crop_0.4_vessel_v2"
target_hospital_dir = "/work/vig/Datasets/aneurysm/hospital/crop_0.4_vessel_edt_v2/"

target_external_vessel_dir = "/work/vig/Datasets/aneurysm/external/crop_0.4_vessel_edt"

train_im_files = sorted(list(glob(f"{train_vessel_dir}/*")))
val_im_files = sorted(list(glob(f"{test_vessel_dir}/*")))
external_im_files = sorted(list(glob(f"{external_vessel_dir}/*")))
hospital_im_files = sorted(list(glob(f"{hospital_vessel_dir}/*")))

In [2]:
file_paths = hospital_im_files
save_dir = target_hospital_dir
# create dir if not exists
Path(save_dir).mkdir(parents=True, exist_ok=True)
for file in tqdm.tqdm(file_paths):
    scan_seriesuid = file.split("/")[-1]
    im_header = sitk.ReadImage(file)
    im = sitk.GetArrayFromImage(im_header)
    im = (im == 1).astype(np.uint8)
    im_dist = edt.sdf(im, black_border=False, parallel=62)
    im_dist_header = sitk.GetImageFromArray(im_dist)
    im_dist_header.CopyInformation(im_header)
    save_file_path = os.path.join(save_dir, scan_seriesuid)
    sitk.WriteImage(im_dist_header, save_file_path)

100%|██████████| 38/38 [1:17:28<00:00, 122.34s/it]


In [5]:
file_paths = val_im_files
save_dir = target_test_vessel_dir

for file in tqdm.tqdm(file_paths):
    scan_seriesuid = file.split("/")[-1]
    im_header = sitk.ReadImage(file)
    im = sitk.GetArrayFromImage(im_header)
    im_dist = edt.sdf(im, black_border=False, parallel=22)
    im_dist_header = sitk.GetImageFromArray(im_dist)
    im_dist_header.CopyInformation(im_header)
    save_file_path = os.path.join(save_dir, scan_seriesuid)
    sitk.WriteImage(im_dist_header, save_file_path)

100%|██████████| 152/152 [1:10:26<00:00, 27.81s/it]


In [7]:
im_header = sitk.ReadImage(file)
im_dist = edt.sdf(im, black_border=False, parallel=22)

im_dist_saved = sitk.ReadImage(save_file_path)
im_dist_saved = sitk.GetArrayFromImage(im_dist_saved)

In [8]:
im_dist.shape, im_dist_saved.shape

((425, 507, 507), (425, 507, 507))

In [10]:
(im_dist - im_dist_saved).min()

0.0

In [ ]:
header = sitk.ReadImage(
    "/work/vig/Datasets/aneurysm/internal_train/crop_0.4_vessel_edt/Tr0188.nii.gz"
)